In [ ]:
# Gerekli Kütüphaneleri Yükleme
print("Installing core libraries (Datasets, Google AI, SentenceTransformers, Scikit-learn)...")
!pip install datasets google-generativeai numpy tqdm sentence-transformers scikit-learn -q
print("✅ Kütüphaneler yüklendi.")

Installing core libraries (Datasets, Google AI, SentenceTransformers, Scikit-learn)...
✅ Kütüphaneler yüklendi.


In [ ]:
# API Yapılandırması ve Gerekli Modülleri İçe Aktarma
import pandas as pd
import numpy as np
import google.generativeai as genai
from google.colab import userdata
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import time
from sklearn.metrics.pairwise import cosine_similarity

print("Configuring the Gemini API...")
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    print("✅ API Configured (for Generation).")
except Exception as e:
    print(f"❌ API Config Error: {e}")
    raise

Configuring the Gemini API...
✅ API Configured (for Generation).


In [ ]:
# SQuAD Veri Setini Yükleme
from datasets import load_dataset
import pandas as pd

print("\nSQuAD veri seti yükleniyor (2000 satır)...")
try:
    dataset = load_dataset("squad", split='train[:2000]') # SQuAD kullan
    df = dataset.to_pandas()

    # SQuAD sütunlarını uyarla
    df.rename(columns={'context': 'description'}, inplace=True)
    df['authors'] = "Wikipedia Contributors"
    df.drop_duplicates(subset=['description'], inplace=True, keep='first')
    df_cleaned = df[['title', 'authors', 'description']].copy()
    df_cleaned.reset_index(drop=True, inplace=True)

    print(f"✅ SQuAD veri seti yüklendi ve temizlendi. {len(df_cleaned)} adet benzersiz doküman işlenecek.")
    display(df_cleaned.head())
except Exception as e:
    print(f"❌ SQuAD yüklenirken hata oluştu: {e}")


Güvenli Yedek Plan: Güvenilir SQuAD veri seti yükleniyor (2000 satır)...


plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

✅ SQuAD veri seti yüklendi ve temizlendi. 266 adet benzersiz doküman işlenecek.


,title,authors,description
0,University_of_Notre_Dame,Wikipedia Contributors,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,Wikipedia Contributors,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,Wikipedia Contributors,The university is the major seat of the Congre...
3,University_of_Notre_Dame,Wikipedia Contributors,The College of Engineering was established in ...
4,University_of_Notre_Dame,Wikipedia Contributors,All of Notre Dame's undergraduate students are...


In [ ]:
# Lokal Embedding Modelini Yükleme
print("\nLoading local embedding model (all-MiniLM-L6-v2)...")
try:
    # 'all-MiniLM-L6-v2' (384 boyutlu) modelini kullanıyoruz
    local_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("✅ Local model loaded.")
except Exception as e:
    print(f"❌ Lokal model yüklenirken hata oluştu: {e}")


Loading local embedding model (all-MiniLM-L6-v2)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Local model loaded.


In [ ]:
# Embedding'leri API'siz (Lokal) Oluşturma
print("Generating embeddings locally using 'all-MiniLM-L6-v2'...")
try:
    # df_cleaned'in bir kopyası üzerinde çalışalım
    df_final = df_cleaned.copy()

    # show_progress_bar=True ile bir ilerleme çubuğu gösteririz.
    embeddings_list = local_model.encode(df_final['description'].tolist(), show_progress_bar=True)

    # Oluşturulan embedding listesini DataFrame'e yeni bir sütun olarak ekle
    df_final['Embeddings'] = list(embeddings_list)

    # Embedding oluşturulamayan satırları temizle
    df_final.dropna(subset=['Embeddings'], inplace=True)

    print("\n✅ Embeddings generated and added to dataframe!")
    display(df_final.head())
except NameError:
     print("❌ Hata: 'df_cleaned' veya 'local_model' bulunamadı. Lütfen önceki adımları (Adım 3 ve 4) çalıştırın.")
except Exception as e:
    print(f"❌ Embedding oluşturulurken hata oluştu: {e}")

Generating embeddings locally using 'all-MiniLM-L6-v2'...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]


✅ Embeddings generated and added to dataframe!


,title,authors,description,Embeddings
0,University_of_Notre_Dame,Wikipedia Contributors,"Architecturally, the school has a Catholic cha...","[0.028794207, 0.14770703, 0.0036130159, 0.0130..."
1,University_of_Notre_Dame,Wikipedia Contributors,"As at most other universities, Notre Dame's st...","[0.06820907, -0.039907645, -0.012214051, 0.023..."
2,University_of_Notre_Dame,Wikipedia Contributors,The university is the major seat of the Congre...,"[-0.010786222, -0.0014746004, -0.06549475, -0...."
3,University_of_Notre_Dame,Wikipedia Contributors,The College of Engineering was established in ...,"[-0.027754072, 0.008923833, -0.00434044, 0.025..."
4,University_of_Notre_Dame,Wikipedia Contributors,All of Notre Dame's undergraduate students are...,"[0.005485516, 0.017865695, 0.077902555, -0.001..."


In [ ]:
# Cevap Kütüphanede Var Mı?

print("\nSearching for the specific answer context ('Denver Broncos') in df_final...")
answer_rows = df_final[df_final['description'].str.contains("Denver Broncos", case=False, na=False)]

if not answer_rows.empty:
    print(f"✅✅✅ SUCCESS: Found {len(answer_rows)} document(s) containing 'Denver Broncos':")
    print("\n--- Example Context with the Answer ---")
    print(answer_rows.iloc[0]['description'])
    print("-------------------------------------")
    print("If you see this, the data exists! The problem is in the search function (Step 6).")
else:
    print("❌❌❌ FAILURE: No documents containing 'Denver Broncos' were found in df_final.")
    print("This means the answer wasn't in the train[:2000] slice after cleaning.")
    print("The 'Couldn't find an answer' response was correct!")


Searching for the specific answer context ('Denver Broncos') in df_final...
❌❌❌ FAILURE: No documents containing 'Denver Broncos' were found in df_final.
This means the answer wasn't in the train[:2000] slice after cleaning.
The 'Couldn't find an answer' response was correct!


In [ ]:
# Arama (Retrieval) Fonksiyonunu Tanımlama (Kosinüs Benzerliği ile)
print("Defining the retrieval function ('find_best_passages_local')...")

# Gerekli kütüphaneler zaten önceki adımlarda import edildi (numpy, sklearn)

def find_best_passages_local(query, dataframe, top_k=3):
    """
    Finds the most relevant passages using the local embedding model
    and COSINE SIMILARITY for better accuracy.
    """
    try:
        # 1. Soruyu lokal model ile embedding'e dönüştür
        query_embedding = local_model.encode(query)

        # 2. Kütüphanedeki tüm embedding'leri numpy array'e dönüştür
        doc_embeddings = np.stack(dataframe['Embeddings'].to_numpy())

        # 3. Kosinüs Benzerliğini Hesapla
        # query_embedding'i 2D array yapmak için reshape kullanılır
        similarities = cosine_similarity(query_embedding.reshape(1, -1), doc_embeddings)[0]

        # 4. En yüksek benzerliğe sahip ilk 'top_k' indeksini bul
        # argsort sıralar, [-top_k:] son k elemanı alır, [::-1] ters çevirir (en yüksekten düşüğe)
        top_indices = np.argsort(similarities)[-top_k:][::-1]

        # 5. Bu indekslerdeki 'description' metinlerini birleştirip döndür
        return "\n---\n".join(dataframe.iloc[top_indices]['description'].tolist())

    except NameError:
         # Eğer local_model veya np tanımlı değilse bu hata alınır
         print("❌ Hata: 'local_model' veya 'np' bulunamadı. Lütfen önceki adımları (Adım 2, 4) çalıştırın.")
         return ""
    except Exception as e:
        print(f"❌ Arama sırasında bir hata oluştu: {e}")
        return ""

print("✅ 'find_best_passages_local' (Arama) fonksiyonu tanımlandı.")

# --- Fonksiyonu küçük bir testle deneyelim (isteğe bağlı) ---
# test_query_retrieval = "What is Notre Dame?"
# test_context = find_best_passages_local(test_query_retrieval, df_final)
# print("\n--- Retrieval Test ---")
# print(f"Query: {test_query_retrieval}")
# print(f"Found Context:\n{test_context[:500]}...") # Sadece ilk 500 karakteri göster

Defining the retrieval function ('find_best_passages_local')...
✅ 'find_best_passages_local' (Arama) fonksiyonu tanımlandı.


In [ ]:
# Arama Sonuçlarını İnceleme

print("\nTesting the retrieval function and examining its raw output with the current library...")
test_query = "Which NFL team represented the AFC at Super Bowl 50?"
try:
    query_embedding = local_model.encode(test_query)
    doc_embeddings = np.stack(df_final['Embeddings'].to_numpy())
    similarities = cosine_similarity(query_embedding.reshape(1, -1), doc_embeddings)[0]
    top_k = 3
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    print(f"\n--- Raw Retrieval Results for query: '{test_query}' ---")
    print(f"Top {top_k} indices found: {top_indices}")
    print("\nSimilarity Scores (higher is better):")
    for i, index in enumerate(top_indices):
        print(f"  Rank {i+1}: Index {index}, Score: {similarities[index]:.4f}")

    print("\nRetrieved Passages Text:")
    retrieved_passages_text = "\n---\n".join(df_final.iloc[top_indices]['description'].tolist())
    print("-------------------- CONTEXT START --------------------")
    print(retrieved_passages_text)
    print("--------------------- CONTEXT END ---------------------")
except Exception as e:
    print(f"❌ An error occurred during retrieval testing: {e}")


Testing the retrieval function and examining its raw output with the current library...

--- Raw Retrieval Results for query: 'Which NFL team represented the AFC at Super Bowl 50?' ---
Top 3 indices found: [39 20 46]

Similarity Scores (higher is better):
  Rank 1: Index 39, Score: 0.3434
  Rank 2: Index 20, Score: 0.3041
  Rank 3: Index 46, Score: 0.3040

Retrieved Passages Text:
-------------------- CONTEXT START --------------------
Notre Dame rose to national prominence in the early 1900s for its Fighting Irish football team, especially under the guidance of the legendary coach Knute Rockne. The university's athletic teams are members of the NCAA Division I and are known collectively as the Fighting Irish. The football team, an Independent, has accumulated eleven consensus national championships, seven Heisman Trophy winners, 62 members in the College Football Hall of Fame and 13 members in the Pro Football Hall of Fame and is considered one of the most famed and successful colleg

In [ ]:
# Cevap Üretme (Generation) Fonksiyonunu Tanımlama
print("Defining the generation function ('generate_answer')...")

def generate_answer(query, dataframe):
    """
    Generates a final answer by combining retrieval (using local model)
    and generation (using Gemini API).
    """
    # Adım 6'da tanımladığımız fonksiyonla en alakalı metinleri bul
    context = find_best_passages_local(query, dataframe)

    # Eğer arama sonucu boş geldiyse (bir hata oluştuysa veya hiç metin bulamadıysa)
    if not context:
        return "I couldn't retrieve relevant information to answer your question."

    # Gemini'a göndereceğimiz komut şablonunu (prompt) oluştur
    prompt = f"""
    You are a helpful and informative bot. Answer the user's QUESTION based ONLY on the CONTEXT provided below.
    Be sure to respond in a complete sentence and be comprehensive.
    If the answer is not explicitly stated in the context, say "I couldn't find an answer in the provided documents."

    CONTEXT:
    {context}

    QUESTION:
    {query}

    ANSWER:
    """

    # Cevap üretmek için Gemini modelini kullan
    try:
        # Daha önce test ettiğimiz ve çalıştığını bildiğimiz model adını kullanıyoruz
        generative_model = genai.GenerativeModel('models/gemini-2.5-flash')
        # Komutu Gemini'a gönder ve cevabı al
        response = generative_model.generate_content(prompt)
        return response.text
    # API ile ilgili bir hata olursa kullanıcıya bilgi ver
    except Exception as e:
        print(f"❌ Cevap üretme sırasında bir API hatası oluştu: {e}")
        # Hatanın detayını da döndürelim (debug için yararlı olabilir)
        return f"An error occurred while generating the answer: {str(e)}"

print("✅ 'generate_answer' (Cevap Üretme) fonksiyonu tanımlandı.")

Defining the generation function ('generate_answer')...
✅ 'generate_answer' (Cevap Üretme) fonksiyonu tanımlandı.


In [ ]:
# Test Etme (Kütüphanedeki Bir Konuyla)
print("\n--- Testing the Full RAG System ---")

# Bu soru artık büyütülmüş (2000 satır) ve doğru oluşturulmuş kütüphanede bulunmalı
query = "Which NFL team represented the AFC at Super Bowl 50?"
# Alternatif test sorusu: query = "What is the University of Notre Dame?"

try:
    # Adım 7'de tanımladığımız ana fonksiyonu çağırıyoruz
    final_answer = generate_answer(query, df_final)

    # Sonucu ekrana yazdırıyoruz
    print(f"\nQuestion: {query}\n")
    print(f"Answer: {final_answer}")

except NameError:
    # Eğer önceki adımlar çalıştırılmadıysa bu hata alınır
    print("❌ Hata: 'df_final' veya 'generate_answer' bulunamadı. Lütfen önceki adımları (Adım 5, 7) çalıştırın.")
except Exception as e:
    # Beklenmedik bir hata olursa
     print(f"❌ Test sırasında bir hata oluştu: {e}")


--- Testing the Full RAG System ---

Question: Which NFL team represented the AFC at Super Bowl 50?

Answer: I couldn't find an answer in the provided documents.


In [27]:
%%writefile requirements.txt
streamlit
google-generativeai
sentence-transformers
pandas
numpy
datasets
scikit-learn
torch

Overwriting requirements.txt


In [34]:
%%writefile app.py

# --- Gerekli Kütüphaneler ---
import streamlit as st
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import os
from streamlit.errors import StreamlitAPIException, StreamlitSecretNotFoundError
from sklearn.metrics.pairwise import cosine_similarity

# --- Sayfa Ayarları ---
st.set_page_config(
    page_title="RAG Bilgi Asistanı",
    page_icon="🧠",
    layout="wide"
)

# --- CSS ile Küçük Dokunuşlar ---
st.markdown("""
<style>
    /* Buton stili (isteğe bağlı) */
    .stButton>button {
        border-radius: 20px;
        border: 2px solid #FF4B4B;
        color: #FF4B4B;
        background-color: transparent;
    }
    .stButton>button:hover {
        border-color: #FF6B6B;
        color: #FF6B6B;
    }

    /* --- YENİ EKLENEN KISIM: Sidebar metin rengini siyah yap --- */
    [data-testid="stSidebar"] {
        background-color: #f0f2f6; /* Kenar çubuğu arka plan rengi (açık tema için) */
    }
    [data-testid="stSidebar"] .stMarkdown p,
    [data-testid="stSidebar"] .stHeader,
    [data-testid="stSidebar"] .stTextArea label,
    [data-testid="stSidebar"] label {
        color: black !important; /* Sidebar içindeki metinleri siyah yap */
    }

</style>
""", unsafe_allow_html=True)

# --- 1. İlk Kurulum ve Modelleri Yükleme ---
@st.cache_resource
def load_models_and_data():
    print("Loading models and data...")
    local_model = SentenceTransformer('all-mpnet-base-v2')
    try:
        df_final = pd.read_pickle("my_rag_library_local.pkl")
    except FileNotFoundError:
        st.error("Kütüphane dosyası (my_rag_library_local.pkl) bulunamadı.")
        return None, None, None
    try:
        api_key = st.secrets["GOOGLE_API_KEY"]
        genai.configure(api_key=api_key)
    except (KeyError, StreamlitSecretNotFoundError, StreamlitAPIException):
        try:
            api_key = os.environ.get('GOOGLE_API_KEY')
            if not api_key: raise ValueError("GOOGLE_API_KEY not found")
            genai.configure(api_key=api_key)
        except Exception as e:
            st.error(f"API Key yapılandırılamadı. Hata: {e}")
            return None, None, None
    generative_model = genai.GenerativeModel('models/gemini-2.5-flash')
    print("Models and data loaded.")
    return local_model, generative_model, df_final

# --- 2. RAG Fonksiyonları ---
def find_best_passages_local(query, dataframe, local_model, top_k=3):
    query_embedding = local_model.encode(query)
    doc_embeddings = np.stack(dataframe['Embeddings'].to_numpy())
    similarities = cosine_similarity(query_embedding.reshape(1, -1), doc_embeddings)[0]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return "\n---\n".join(dataframe.iloc[top_indices]['description'].tolist())

def generate_answer(query, dataframe, local_model, generative_model):
    context = find_best_passages_local(query, dataframe, local_model)
    prompt = f"""
    You are a helpful bot. Answer the user's QUESTION based ONLY on the CONTEXT provided.
    If the answer is not in a context, say "I couldn't find an answer in the provided documents."
    CONTEXT: {context}
    QUESTION: {query}
    ANSWER:
    """
    try:
        response = generative_model.generate_content(prompt)
        clean_response = response.text.strip()
        return clean_response
    except Exception as e:
        return f"Cevap üretme sırasında bir hata oluştu: {e}"

# --- 3. Streamlit Arayüz Kodu ---
st.markdown("<h1 style='text-align: center; color: #FF4B4B;'>🧠 RAG Bilgi Asistanı</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center;'>SQuAD veri setini kullanan RAG tabanlı bir chatbot demosu</p>", unsafe_allow_html=True)
st.divider()

local_model, generative_model, df_final = load_models_and_data()

with st.sidebar:
    st.image("https://streamlit.io/images/brand/streamlit-logo-secondary-colormark-darktext.svg", width=200)
    st.header("❓ Soru Sor")
    st.markdown("SQuAD veri setindeki (Wikipedia makaleleri) bilgilere dayanarak sorularınızı yanıtlamaya çalışırım.")
    user_query = st.text_area("Sorunuz:", placeholder="Which NFL team won Super Bowl 50?", height=100)
    submit_button = st.button("🚀 Cevap Al")

if local_model and generative_model and (df_final is not None):
    if submit_button:
        if user_query:
            answer_placeholder = st.empty()
            answer_placeholder.info("⏳ Cevabınız aranıyor ve oluşturuluyor...")
            answer = generate_answer(user_query, df_final, local_model, generative_model)
            answer_placeholder.empty()
            st.subheader("🤖 Asistanın Cevabı:")
            st.markdown(f"> {answer}")
        else:
            st.warning("⚠️ Lütfen bir soru girin.")
else:
    st.error("❌ Modeller veya veri yüklenemediği için uygulama başlatılamadı.")

Overwriting app.py
